In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path
from PIL import Image
from scipy.signal import find_peaks

In [2]:
train_dataset = Path('../datasets/train')

In [15]:
def transform(image_array):
    gray = Image.fromarray(image_array.reshape(32, 32), mode='L')

    hist, _ = np.histogram(np.array(gray), bins=257, range=(-1, 255))
    peaks, _ = find_peaks(hist, height=15, width=3, distance=20)

    threshold = np.mean(peaks)
    bw = gray.point(lambda x: 255 if x > threshold else 0)

    bw_array = np.array(bw).astype('float32')
    return np.expand_dims(bw_array, -1)


In [16]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=transform
)


In [17]:
train_generator = datagen.flow_from_directory(
    train_dataset,
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)


Found 50 images belonging to 2 classes.


In [18]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 1)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.legacy.Adam(0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)


In [19]:
model.fit(train_generator, epochs=10)

Epoch 1/10


/Users/jachym.solecky/.pyenv/versions/3.9.16/envs/ml/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jachym.solecky/.pyenv/versions/3.9.16/envs/ml/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2/2 [==============================] - 0s 32ms/step - loss: 0.7135 - accuracy: 0.5400
Epoch 2/10
2/2 [==============================] - 0s 25ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 0s 26ms/step - loss: 0.6962 - accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 28ms/step - loss: 0.6997 - accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 26ms/step - loss: 0.6955 - accuracy: 0.4000
Epoch 6/10
2/2 [==============================] - 0s 51ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 47ms/step - loss: 0.6921 - accuracy: 0.5600
Epoch 8/10
2/2 [==============================] - 0s 25ms/step - loss: 0.6942 - accuracy: 0.4800
Epoch 9/10
2/2 [==============================] - 0s 27ms/step - loss: 0.6926 - accuracy: 0.5200
Epoch 10/10
2/2 [==============================] - 0s 47ms/step - loss: 0.6982 - accuracy: 0.4600
